## 3.3.Decision Tree

In [1]:
import pandas as pd 
df_train_data = pd.read_csv('final_train_data_StdSc.csv')
df_test_data=pd.read_csv('final_test_data_StdSc.csv')

In [2]:
# RFE20
target_train = df_train_data[['is_churn']]

data_train = df_train_data[['registered_via_9','registered_via_4','registered_via_3','avg_actual_amount_paid','avg_plan_list_price','active_days','total_cancel','is_auto_renew','most_fq_payment_method_id','number_of_days_lastthree_listened','number_of_days_lasttwo_listened','num_totalsec_lasttwo_mean','num_100_lasttwo_mean','num_unq_lasttwo_sum','number_of_days_201701_listened','num_unq_201701_sum','number_of_days_201702_listened','num_totalsec_201702_mean','num_totalsec_201702_sum','number_of_days_listened',]]

target_test = df_test_data[['is_churn']]

data_test =df_test_data[['registered_via_9','registered_via_4','registered_via_3','avg_actual_amount_paid','avg_plan_list_price','active_days','total_cancel','is_auto_renew','most_fq_payment_method_id','number_of_days_lastthree_listened','number_of_days_lasttwo_listened','num_totalsec_lasttwo_mean','num_100_lasttwo_mean','num_unq_lasttwo_sum','number_of_days_201701_listened','num_unq_201701_sum','number_of_days_201702_listened','num_totalsec_201702_mean','num_totalsec_201702_sum','number_of_days_listened',]]


In [3]:
# GridSearch: tunning parameter
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# Set the parameters by cross-validation
parameters = [{"criterion": ["gini", "entropy"],
              "min_samples_split": [2, 10, 20],
              "max_depth": [None, 5, 10, 15],
              "min_samples_leaf": [1, 5, 10],
              "max_leaf_nodes": [None, 5, 10, 20],
              }]

tree = GridSearchCV(DecisionTreeClassifier(), parameters, cv=5)
tree.fit(data_train, target_train)

print(tree.best_params_)

{'criterion': 'entropy', 'max_depth': 10, 'max_leaf_nodes': None, 'min_samples_leaf': 10, 'min_samples_split': 10}


In [7]:
# apply decision tree model
from sklearn.tree import DecisionTreeClassifier
decision_tree = DecisionTreeClassifier(criterion='entropy',min_impurity_decrease =0.00015,max_depth=15, max_leaf_nodes=None, min_samples_leaf=10, min_samples_split=10)
decision_tree.fit(data_train, target_train)

prediction = decision_tree.predict(data_test)

# Performance evaluation

from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score

def confusion_matrix_report(y_true, y_pred):
    cm, labels = confusion_matrix(y_true, y_pred), unique_labels(y_true, y_pred)
    column_width = max([len(str(x)) for x in labels] + [5])  # 5 is value length
    report = " " * column_width + " " + "{:_^{}}".format("Prediction", column_width * len(labels))+ "\n"
    report += " " * column_width + " ".join(["{:>{}}".format(label, column_width) for label in labels]) + "\n"
    for i, label1 in enumerate(labels):
        report += "{:>{}}".format(label1, column_width) + " ".join(["{:{}d}".format(cm[i, j], column_width) for j in range(len(labels))]) + "\n"
    return report

def loggloss(target_test, model, data_test):
    probabilities=model.predict_proba(data_test)
    value=log_loss(target_test, probabilities)
    return value

def AUC(target_test, model, data_test):
    values=model.predict_proba(data_test)[:,1]
    auc_score=roc_auc_score(target_test, values)
    return auc_score
    
def analytics(target_test, model, data_test):#target of the test data #predictions as 0,1 #model (knnclassifier) #data_test
    y_pred=model.predict(data_test)
    print("Confusion Matrix:")
    print(confusion_matrix_report(target_test,prediction))
    print("Accuracy Score:")
    print(accuracy_score(target_test,prediction))
    print()
    print("Classification Report:")
    print(classification_report(target_test,prediction))
    print("Log Loss:")
    print(loggloss(target_test, model, data_test))
    print()
    print("AUC Score:")
    print(AUC(target_test, model, data_test))
    
analytics(target_test, decision_tree, data_test)


Confusion Matrix:
      Prediction
         0     1
    0220376 25174
    1 1435 15858

Accuracy Score:
0.898764661794

Classification Report:
             precision    recall  f1-score   support

          0       0.99      0.90      0.94    245550
          1       0.39      0.92      0.54     17293

avg / total       0.95      0.90      0.92    262843

Log Loss:
0.198808133467

AUC Score:
0.969858377636
